In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
import data
from models import TestModels
import moveFiles
from keras.models import load_model, Model
import keras
import pandas as pd

Using TensorFlow backend.


In [3]:
file_limit = 40
train, test, filenames = moveFiles.moveFiles(limit_files=file_limit)

pp = data.Preprocessing()
pp.extractAllVideos()

In [4]:
dataset = data.DataSet()
dataset.dumpNumpyFiles( seq_len_limit=20)

  0%|          | 0/27 [00:00<?, ?it/s]

Exporting Train Data...


  0%|          | 0/7 [00:00<?, ?it/s]

Exporting Test Data


100%|██████████| 7/7 [00:04<00:00,  1.75it/s]


In [3]:
x,y,yseq = dataset.all_data_from_npz('train')

In [696]:
x.shape

(13, 20, 224, 224, 3)

In [4]:
tm3d = TestModels(5, 'c3d')
tm3d.model.fit(x,y, epochs=20, shuffle=True)

Epoch 1/20
13/13 [==============================] - 68s 5s/step - loss: 1.5306 - acc: 0.1538
Epoch 2/20
13/13 [==============================] - 140s 11s/step - loss: 9.4025 - acc: 0.3846
Epoch 3/20
13/13 [==============================] - 140s 11s/step - loss: 5.7784 - acc: 0.5385
Epoch 4/20
13/13 [==============================] - 132s 10s/step - loss: 7.0482 - acc: 0.3077
Epoch 5/20
13/13 [==============================] - 235s 18s/step - loss: 4.8796 - acc: 0.5385
Epoch 6/20
13/13 [==============================] - 176s 14s/step - loss: 5.4330 - acc: 0.5385
Epoch 7/20


KeyboardInterrupt: 

In [77]:
tm = TestModels(5, 'lrcn')
tm.model.fit(x,yseq, epochs=5, shuffle=True)

Epoch 1/5
13/13 [==============================] - 11s 879ms/step - loss: 1.5550 - acc: 0.3269
Epoch 2/5
13/13 [==============================] - 9s 705ms/step - loss: 1.4318 - acc: 0.6154
Epoch 3/5
13/13 [==============================] - 9s 656ms/step - loss: 1.3303 - acc: 0.7423
Epoch 4/5
13/13 [==============================] - 8s 616ms/step - loss: 1.2748 - acc: 0.7885
Epoch 5/5
13/13 [==============================] - 8s 619ms/step - loss: 1.2094 - acc: 0.7962


In [78]:
tm.model.save('test-model.hdf5')
m = load_model('test-model.hdf5')

# Create Model to Pull 'Dense' Features

In [10]:
dense_m = Model(inputs=m.input, outputs=m.get_layer('fc1').output)

## Convert Data into the Format for LSTM Model

In [11]:
x_lstm = [np.array([x[i]]) for i in range(len(x))]
x_lstm = np.array([dense_m.predict(k) for k in x_lstm])
x_lstm = x_lstm.squeeze(axis=1) #squeeze to get into the right shape for LSTM
y_lstm = np.array([np.array([yseq[i]]) for i in range(len(yseq))])
y_lstm = y_lstm.squeeze(axis=1)

In [40]:
lstm = TestModels(5, 'lstm')
lstm.model.fit(x_lstm, y_lstm, epochs=1, batch_size=1)

Epoch 1/1
13/13 [==============================] - 1s 55ms/step - loss: 1.6274 - acc: 0.2192


In [523]:
preds

array([[[0.15772662, 0.22016934, 0.21478514, 0.22600219, 0.18131666],
        [0.13737012, 0.24336159, 0.23757575, 0.22889659, 0.15279596],
        [0.14084533, 0.26827225, 0.2349611 , 0.2277287 , 0.12819263],
        [0.1336434 , 0.2648916 , 0.23738754, 0.2371517 , 0.12692577],
        [0.12384777, 0.25966668, 0.2464062 , 0.23897444, 0.13110504],
        [0.1216127 , 0.27621013, 0.25931722, 0.21729437, 0.12556559],
        [0.12350351, 0.30517232, 0.25629488, 0.20074528, 0.11428399],
        [0.17719302, 0.36703503, 0.20810172, 0.16570716, 0.081963  ],
        [0.15881833, 0.34934646, 0.20629573, 0.1782313 , 0.1073081 ],
        [0.13887618, 0.36800525, 0.22086538, 0.17095467, 0.10129861],
        [0.13666183, 0.3827532 , 0.21830082, 0.16733171, 0.0949525 ],
        [0.13160451, 0.39341813, 0.21615763, 0.16510563, 0.09371403],
        [0.12405361, 0.39553115, 0.21653631, 0.1624374 , 0.1014416 ],
        [0.10604483, 0.3785264 , 0.2427527 , 0.16279788, 0.10987824],
        [0.10200866,

In [28]:
#Takes LSTM predictions and generates sequences by assuming all 'normal' unless there is a prediction with a
#value higher than threshold. 
def seqFromPreds(preds, threshold=0.35):
    preds = preds.squeeze()
    seq = ['normal']*20
    
    for i in range(len(preds)):
        p = preds[i]
        m = np.max(p)
        a = np.argmax(p)
        if m>=threshold:
            seq[i]=dataset.classes[a]
    return seq

def seqOverlap(df):
    test = df[df.actual != 'normal']
    match = test[df.actual == df.predicted]
    total = len(test)*1.0
    matches = len(match)*1.0
    return (matches/total)
    
    

In [122]:
n = 12
x1 = np.array([x[n]])
x1 = dense_m.predict(x1)
y1 = np.array([yseq[n]])

In [125]:
preds = m.predict(np.array([x[n]])).squeeze()
predSeq = [dataset.classes[p.argmax()] for p in preds]
actualSeq = [dataset.reverse_one_hot(k) for k in y1.squeeze()]
df = pd.DataFrame({'actual': actualSeq, 'predicted': predSeq})
df

,actual,predicted
0,normal,normal
1,normal,normal
2,normal,normal
3,normal,normal
4,normal,normal
5,normal,normal
6,normal,normal
7,normal,normal
8,dropped,normal
9,normal,normal


In [681]:
seqOverlap(df)

/Users/matt/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


0.7

In [82]:

convPred = []
for i in range(20):
    convPred.append(m.predict(np.array([[x[n][i]]])))
convPred = np.array(convPred)

In [83]:
print(convOverall)
seqFromPreds(convPred, 0.0)

compressed


['normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal']

In [85]:
# convOverall = dataset.classes[m.predict(np.array([x[n]])).argmax()]
# print(convOverall)
overalls = []
actuals = []
for i in range(len(x)):
    overall = dataset.classes[m.predict(np.array([x[i]])).argmax()]
    actual = dataset.reverse_one_hot(y[i])
    overalls.append(overall)
    actuals.append(actual)
dfConv = pd.DataFrame({'actual': actuals, 'convOverallPrediction': overalls})
dfConv

IndexError: list index out of range

['normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal']